In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from keras.models import load_model
from PyQt5.QtWidgets import QMessageBox
import numpy as np
from numpy import argmax ,asarray
import pickle
import cv2 

class CNN_Model():
    X = [] #for Input features of Train Data
    y = [] #for Labels of Train Data
    VAl_X = [] #for Input features of Validation_data
    VAl_y = [] #for Labels of Validation_data
    train_datagen = ImageDataGenerator(rotation_range=15,shear_range=0.15,zoom_range=0.15,horizontal_flip=True,)

    
    def load_Data(self):

        pickle_in = open("SavedDataSet\X.pickle","rb")
        self.X = pickle.load(pickle_in)

        pickle_in = open("SavedDataSet\y.pickle","rb")
        self.y = pickle.load(pickle_in)

        pickle_in = open("SavedDataSet\VAl_X.pickle","rb")
        self.VAl_X = pickle.load(pickle_in)

        pickle_in = open("SavedDataSet\VAl_y.pickle","rb")
        self.VAl_y = pickle.load(pickle_in)
               
        self.train_datagen.fit(self.X)

        
        
    def Create_Train_Model(self):
        
    #-----------model definition----------------
    #-------------create the model attributes-------------------
        model = Sequential()
        model.add(Conv2D(32, (5, 5), activation='relu',padding='same',input_shape=(48, 48, 1)))
        model.add(BatchNormalization())

        model.add(Conv2D(64, (5, 5), activation='relu',padding='same'))    
        model.add(BatchNormalization()) 
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2,2)))


        model.add(Conv2D(256, (3, 3), activation='relu',padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.2))

        model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

        model.add(Dense(128, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

        model.add(Dense(6, activation='softmax'))

            #-----------set model compilation attributes------------------------
        model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

            #--------to just see  the Model detials-----------------------------
        model.summary()
        #-------train the model on data set---------------------------------
        
        history = model.fit(self.train_datagen.flow(self.X, self.y, batch_size=32),epochs=60,steps_per_epoch=1000,validation_data=(self.VAl_X,self.VAl_y),shuffle=True)
    #-------------------Save the model on device------------------------
        print("create and train model successfully.")
       # -------------------Save the model on device------------------------
        print("Saving model....")
        model.save("U_Model.h5")
        print("model saved sucessfully.")
        print(" ")
        print("Evaluate model on validation data...")
        self.evaluate_Model(model,history)
        print("done.")
        
    def evaluate_Model(self,model,history):
    #-----------------------evaluate the model to show the accuracy and losses-----------
      
        _, train_acc = model.evaluate(self.X,self.y)
        _, test_acc = model.evaluate(self.VAl_X, self.VAl_y)
        print('Train: %.3f, Test: %.3f' % (train_acc,test_acc))

        #-------------------------plot losses during training-----------------------------
        pyplot.subplot(211)
        pyplot.title('Loss')
        pyplot.plot(history.history['loss'], label='train')
        pyplot.plot(history.history['val_loss'], label='test')
        pyplot.legend()
        #-------------------------plot accuracy during training-----------------------------
        pyplot.subplot(212)
        pyplot.title('Accuracy')
        pyplot.plot(history.history['accuracy'], label='train')
        pyplot.plot(history.history['val_accuracy'], label='test')
        pyplot.legend()
        pyplot.show()
        
        
 #---------------model prediction------------------------------------------------------

    def Model_prediction(self,filename):
    #--------------Load the saved model from file
        model = load_model('U_Model.h5')
        CATEGORIES = ["angry","disgust","fear","happy","sad","surprise"]
        model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
                
        #-----------Read and reshape the Image for prediction-----------------------
        print(filename)

        Original_img = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
        #Original_img = cv2.imread('test/Sad.jpg',cv2.IMREAD_GRAYSCALE)
        
        laplacian =cv2.Laplacian(Original_img,cv2.CV_64F)
        img=Original_img-laplacian

        img = cv2.resize(img,(48,48))
        img = img.reshape((img.shape[0], img.shape[1], 1))
        img = img.astype('float32') / 255.0
        classes = model.predict(np.asarray([img]))
        #-----------------print the predicted class for given image-------------------------------
        print('Predicted: class=%d' % np.argmax(classes))
        y_classes = classes.argmax(axis=1)
        #print(CATEGORIES[int(y_classes)])
        return CATEGORIES[int(y_classes)]
        #----------------------------------------------------------------------------------------

    def update_model(self):

        print("Loading Data...")
        self.load_Data()
        print("done.")
        print("Create and Fit model....")
        self.Create_Train_Model()
        print("done..")
        
        msgBox = QMessageBox()
        msgBox.setIcon(QMessageBox.Information)
        msgBox.setText("Update Model succesfully")
        msgBox.setWindowTitle("Update Model")
        msgBox.setStandardButtons(QMessageBox.Ok)
        
        returnValue = msgBox.exec()
        
        
#cnn=CNN_Model()
#cnn.Model_prediction('test images/fear.jpg')
#cnn.update_model()


test images/fear.jpg
Predicted: class=2


'fear'